Python Assignment 

part - I

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep

In [2]:
headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36'}

In [3]:
baselink = 'https://www.amazon.in/'

In [4]:
searchquery = 'bags'.replace(' ','+')
baseurl = 'https://www.amazon.in/s?k={0}'.format(searchquery)

In [ ]:
items=[] #taking all the columns values into this list
asins=[] #taking asin numbers which gives unique values of each product

for i in range(1,51): #scarping the data of 50 pages
    url = "https://www.amazon.in/s?k=bags&crid=2M096C61O4MLT&qid=1653308124&sprefix=ba%2Caps%2C283&ref=sr_pg_"+str(i)
    
    response=requests.get(url,headers=headers)
    
    soup = BeautifulSoup(response.content,'html.parser')
    
    results = soup.find_all('div',{'class':'s-result-item','data-component-type':'s-search-result'})
    
    for result in results:
        productname = result.h2.text  # extracting the product name
        asinno = result.attrs['data-asin'] # extracting asin numbers
        
        asins.append(asinno)
        
        # finding out the count of reviews
        
        reviewurl = 'https://www.amazon.in/product-reviews/{0}/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews'.format(asinno)
        
        req = requests.get(reviewurl,headers=headers)
        
        soupreview=BeautifulSoup(req.content,'html.parser')
        
        # finding out the price of product 
        
        priceurl = "https://www.amazon.in/dp/{0}/ref=sr_1_1_sspa?crid=2M096C61O4MLT&keywords=bags&qid=1663926193&sprefix=ba%2Caps%2C283&sr=8-1-spons&psc=1".format(asinno)
        
        reqprice = requests.get(priceurl,headers=headers)
        
        soupprice = BeautifulSoup(reqprice.content,'html.parser')
        
        try:
            review= soupreview.find('div',{'class':'a-row a-spacing-base a-size-base'}).text.split('ratings,',1)[1].strip().replace('with reviews','')
            
        except AttributeError:
            
            continue
            
        try:
            rating = result.find('span',{'class':'a-size-base'}).text
            
        except AttributeError:
            
            continue
            
        try:
            price1 = soupprice.find('span',{'class':'a-price-whole'}).text.replace(',','')
            price2= soupprice.find('span',{'class':'a-price-fraction'}).text
            price = float(price1+price2)
            producturl = 'https://amazon.com'+result.h2.a['href']
            
        except AttributeError:
            
            continue
            
        items.append([producturl,productname,price,rating,review])
        
    sleep(1.5)

In [ ]:
df = pd.DataFrame(items,columns=['producturl','productnames','prices','ratings','reviews'])
df.to_csv('part-I.csv',index=False)